In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv

In [ ]:
class Stem(keras.layers.Layer):
    def __init__(self, f):
        keras.layers.Conv2D()

In [ ]:
class InceptionModule(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        filters_1x1 = filters[0:2] + filters[3:4] + filters[5:]
        filters_3x3 = filters[2]
        filters_5x5 = filters[4]
        self.conv1x1_1 = keras.layers.Conv2D(filters_1x1[0], kernel_size=1, strides=1, padding="same")
        self.conv1x1_2 = keras.layers.Conv2D(filters_1x1[1], kernel_size=1, strides=1, padding="same")
        self.conv1x1_3 = keras.layers.Conv2D(filters_1x1[2], kernel_size=1, strides=1, padding="same")
        self.conv1x1_4 = keras.layers.Conv2D(filters_1x1[3], kernel_size=1, strides=1, padding="same")

        self.conv3 = keras.layers.Conv2D(filters_3x3, kernel_size=3, strides=1, padding="same")
        self.conv5 = keras.layers.Conv2D(filters_5x5, kernel_size=3, strides=1, padding="same")

        self.max_pool = keras.layers.MaxPooling2D(pool_size=3, strides=1, padding="same")

    def call(self, inputs):
        Z_1 = self.conv1x1_1(inputs)

        Z_2 = self.conv1x1_2(inputs)
        Z_2 = self.conv3(Z_2)

        Z_3 = self.conv1x1_3(inputs)
        Z_3 = self.conv5(Z_3)

        Z_4 = self.max_pool(inputs)
        Z_4 = self.conv1x1_4(Z_4)

        return keras.layers.Concatenate(axis=-1)([Z_1, Z_2, Z_3, Z_4])

In [3]:
class ResNetUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, padding="samen", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, kernel_size=1, strides=strides, padding="same", use_bias=False),
                keras.layers.BatchNormalization()
            ]


    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        
        return self.activation(skip_Z + Z)

In [ ]:
class SE_Block(keras.layers.Layer):
    def __init__(self, filters, ratio=16):
        self.global_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.squeeze = keras.layers.Dense(filters//ratio, activation='relu')
        self.excitation = keras.layers.Dense(filters, activation='sigmoid')
    
    def call(self, inputs):
        Z = self.global_avg_pool(inputs)
        Z = self.squeeze(Z)
        return self.excitation(Z)

In [ ]:
class XceptionModule(keras.layers.Layer):
    def __init__(self, filter):
        self.depthwise = keras.layers.Conv2D(filter, kernel_size=3, strides=1, padding='same')
        self.pointwise = keras.layers.Conv2D(filter, kernel_size=1, strides=1, padding="same")
    
    def call(self, inputs):
        Z = inputs
        Z = self.depthwise(Z)
        Z = keras.layers.BatchNormalization()(Z)
        Z = self.pointwise(Z)
        return keras.layers.BatchNormalization()(Z)